# K-Means: Шпаргалка
Корисні блоки коду для кластеризації K-Means

## 📦 Імпорти

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
    silhouette_samples
)

## 🎲 Генерація тестових даних

In [ ]:
# Синтетичні дані з кластерами
X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)

# Візуалізація
plt.scatter(X[:, 0], X[:, 1], alpha=0.6)
plt.title('Згенеровані дані')
plt.show()

## ⚙️ Базове використання K-Means

In [ ]:
# Створення та навчання моделі
kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(X)

# Результати
print(f"Центроїди:\n{kmeans.cluster_centers_}")
print(f"Інерція: {kmeans.inertia_:.2f}")
print(f"Кількість ітерацій: {kmeans.n_iter_}")

# Візуалізація
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', alpha=0.6)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
           c='red', marker='X', s=300, edgecolors='black', linewidths=2)
plt.title('K-Means кластеризація')
plt.show()

## 📏 Масштабування даних (важливо!)

In [ ]:
# K-Means чутливий до масштабу - завжди масштабуй!
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Кластеризація на масштабованих даних
kmeans = KMeans(n_clusters=4, random_state=42)
labels = kmeans.fit_predict(X_scaled)

# Порівняння якості
labels_no_scale = KMeans(n_clusters=4, random_state=42).fit_predict(X)
print(f"Без масштабування: {silhouette_score(X, labels_no_scale):.3f}")
print(f"З масштабуванням:  {silhouette_score(X_scaled, labels):.3f}")

## 📊 Метод ліктя (Elbow Method)

In [ ]:
# Тестування різних K
K_range = range(1, 11)
inertias = []

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)

# Графік
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'o-', linewidth=2, markersize=8)
plt.xlabel('Кількість кластерів (K)')
plt.ylabel('Інерція')
plt.title('Метод ліктя')
plt.grid(True, alpha=0.3)
plt.show()

## 🎯 Автоматичний пошук "ліктя"

In [ ]:
# pip install kneed
from kneed import KneeLocator

kl = KneeLocator(K_range, inertias, curve='convex', direction='decreasing')
optimal_k = kl.elbow
print(f"Оптимальний K: {optimal_k}")

## 🔍 Silhouette Score

In [ ]:
# Обчислення для різних K
K_range = range(2, 11)
silhouette_scores = []

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X)
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)
    print(f"K={k}: {score:.3f}")

# Графік
plt.figure(figsize=(10, 6))
plt.plot(K_range, silhouette_scores, 'o-', linewidth=2, markersize=8)
plt.xlabel('K')
plt.ylabel('Silhouette Score')
plt.title('Silhouette для різних K (чим вище, тим краще)')
plt.axhline(y=0.5, color='red', linestyle='--', alpha=0.5)
plt.grid(True, alpha=0.3)
plt.show()

optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"\nОптимальний K: {optimal_k}")

## 📈 Silhouette Plot (детальна візуалізація)

In [ ]:
import matplotlib.cm as cm

def plot_silhouette(X, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X)
    
    silhouette_avg = silhouette_score(X, labels)
    sample_silhouette_values = silhouette_samples(X, labels)
    
    fig, ax = plt.subplots(figsize=(10, 7))
    y_lower = 10
    
    for i in range(n_clusters):
        cluster_values = sample_silhouette_values[labels == i]
        cluster_values.sort()
        
        size = cluster_values.shape[0]
        y_upper = y_lower + size
        
        color = cm.nipy_spectral(float(i) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_values,
                        facecolor=color, edgecolor=color, alpha=0.7)
        ax.text(-0.05, y_lower + 0.5 * size, str(i))
        y_lower = y_upper + 10
    
    ax.set_title(f'Silhouette Plot (K={n_clusters})')
    ax.set_xlabel('Silhouette коефіцієнт')
    ax.set_ylabel('Кластер')
    ax.axvline(x=silhouette_avg, color="red", linestyle="--",
              label=f'Середнє: {silhouette_avg:.3f}')
    ax.set_yticks([])
    ax.set_xlim([-0.1, 1])
    ax.legend()
    plt.show()

# Використання
plot_silhouette(X, n_clusters=4)

## 📊 Всі метрики разом

In [ ]:
def evaluate_kmeans(X, k_range):
    results = []
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X)
        
        results.append({
            'K': k,
            'Inertia': kmeans.inertia_,
            'Silhouette': silhouette_score(X, labels),
            'Davies-Bouldin': davies_bouldin_score(X, labels),  # мінімум краще
            'Calinski-Harabasz': calinski_harabasz_score(X, labels)  # максимум краще
        })
    
    return pd.DataFrame(results)

# Оцінка
results_df = evaluate_kmeans(X, range(2, 11))
print(results_df.to_string(index=False))

## 📉 Візуалізація всіх метрик

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Inertia (мінімум)
axes[0, 0].plot(results_df['K'], results_df['Inertia'], 'o-')
axes[0, 0].set_title('Inertia (шукаємо "лікоть")')
axes[0, 0].set_xlabel('K')
axes[0, 0].grid(True, alpha=0.3)

# Silhouette (максимум)
axes[0, 1].plot(results_df['K'], results_df['Silhouette'], 'o-', color='green')
axes[0, 1].set_title('Silhouette (↑ краще)')
axes[0, 1].set_xlabel('K')
axes[0, 1].grid(True, alpha=0.3)

# Davies-Bouldin (мінімум)
axes[1, 0].plot(results_df['K'], results_df['Davies-Bouldin'], 'o-', color='red')
axes[1, 0].set_title('Davies-Bouldin (↓ краще)')
axes[1, 0].set_xlabel('K')
axes[1, 0].grid(True, alpha=0.3)

# Calinski-Harabasz (максимум)
axes[1, 1].plot(results_df['K'], results_df['Calinski-Harabasz'], 'o-', color='purple')
axes[1, 1].set_title('Calinski-Harabasz (↑ краще)')
axes[1, 1].set_xlabel('K')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## ⚡ MiniBatch K-Means (для великих даних)

In [ ]:
import time

# Великий датасет
X_large, _ = make_blobs(n_samples=100000, centers=5, random_state=42)

# Звичайний K-Means
start = time.time()
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X_large)
time_kmeans = time.time() - start

# MiniBatch K-Means
start = time.time()
mbkmeans = MiniBatchKMeans(n_clusters=5, batch_size=1000, random_state=42)
mbkmeans.fit(X_large)
time_mbkmeans = time.time() - start

print(f"KMeans:          {time_kmeans:.2f} сек")
print(f"MiniBatchKMeans: {time_mbkmeans:.2f} сек")
print(f"Прискорення:     {time_kmeans/time_mbkmeans:.1f}x")

# Якість
labels_km = kmeans.predict(X_large)
labels_mbkm = mbkmeans.predict(X_large)
print(f"\nSilhouette KMeans:    {silhouette_score(X_large, labels_km):.3f}")
print(f"Silhouette MiniBatch: {silhouette_score(X_large, labels_mbkm):.3f}")

## 🎬 Покрокова анімація алгоритму

In [ ]:
# Ручна реалізація для візуалізації
np.random.seed(42)
initial_centers = X[np.random.choice(X.shape[0], 4, replace=False)]

history_centers = [initial_centers.copy()]
history_labels = []

centers = initial_centers.copy()
for iteration in range(10):
    # Призначення до кластерів
    distances = np.sqrt(((X - centers[:, np.newaxis])**2).sum(axis=2))
    labels = np.argmin(distances, axis=0)
    history_labels.append(labels.copy())
    
    # Оновлення центроїдів
    new_centers = np.array([X[labels == i].mean(axis=0) for i in range(4)])
    
    if np.allclose(centers, new_centers):
        break
    
    centers = new_centers
    history_centers.append(centers.copy())

print(f"Збіглося за {len(history_centers)-1} ітерацій")

In [ ]:
# Візуалізація ітерацій (виберіть iteration від 0 до кінця)
iteration = 0  # змінюй це значення

plt.figure(figsize=(10, 6))

if iteration < len(history_labels):
    labels = history_labels[iteration]
    plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', alpha=0.6, s=50)
else:
    plt.scatter(X[:, 0], X[:, 1], color='gray', alpha=0.6, s=50)

centers = history_centers[iteration]
plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='X', s=300,
           edgecolors='black', linewidths=2, label='Центроїди')

plt.title(f'Ітерація {iteration}')
plt.xlabel('Ознака 1')
plt.ylabel('Ознака 2')
plt.legend()
plt.show()

## 💡 Швидкі поради

**Вибір K:**
- Метод ліктя (Elbow) - шукай "лікоть" на графіку інерції
- Silhouette Score - максимізуй (>0.5 добре)
- Davies-Bouldin - мінімізуй
- Calinski-Harabasz - максимізуй

**Попередня обробка:**
- ✅ Завжди масштабуй дані (StandardScaler)
- ✅ Видаляй викиди
- ✅ Обирай релевантні ознаки

**Параметри:**
- `n_init=10` - кількість запусків з різними початковими центрами
- `max_iter=300` - максимум ітерацій
- `random_state=42` - для відтворюваності

**Для великих даних:**
- Використовуй MiniBatchKMeans
- Зменши `batch_size` якщо бракує пам'яті
- Або спочатку відбери вибірку даних